In [10]:
import simpy
import random
import numpy as np

In [11]:
# Parámetros de la simulación
NUM_SERVIDORES = 5                  # Número de servidores
TIEMPO_PROCESAMIENTO_PROM = 0.1     # Tiempo promedio de procesamiento en segundos
TIEMPO_BALANCEADOR = 0.01           # Tiempo que toma al balanceador derivar la solicitud en segundos
TASA_SOLICITUDES = 5                # Tasa de llegada de solicitudes en solicitudes/segundo (poisson)
UMBRAL_PROPAGACION = 2              # Diferencia de carga para propagación en balanceador difuso
TIEMPO_SIMULACION = 100             # Tiempo total de simulación en segundos

# Inicialización de métricas
tiempos_espera_cola = []
tiempos_atencion_servidores = []
tiempos_espera_balanceador = []

# Definición de la red de servidores en forma de grafo (lista de adyacencia)
grafo = {
    0: [1, 2],
    1: [0, 3, 4],
    2: [0, 3],
    3: [1, 2, 4],
    4: [1, 3]
}

In [12]:
class Servidor:
    def __init__(self, env, id_servidor):
        self.env = env
        self.id_servidor = id_servidor
        self.procesador = simpy.Resource(env, capacity=1)
        self.carga = 0

    def procesar(self, solicitud):
        # el tiempo de procesamiento del servidor sigue una distribución exponencial
        tiempo_procesamiento = random.expovariate(1 / TIEMPO_PROCESAMIENTO_PROM) 
        self.carga += 1
        yield self.env.timeout(tiempo_procesamiento)
        tiempos_atencion_servidores.append(tiempo_procesamiento)
        self.carga -= 1

In [13]:
class BalanceadorRoundRobin:
    def __init__(self, env, servidores):
        self.env = env
        self.servidores = servidores
        self.ultimo_servidor = 0

    def balancear(self, solicitud):
        tiempo_inicio_balanceo = self.env.now
        yield self.env.timeout(TIEMPO_BALANCEADOR)
        tiempos_espera_balanceador.append(self.env.now - tiempo_inicio_balanceo)

        servidor = self.seleccionar_servidor()
        with servidor.procesador.request() as req:
            tiempo_inicio_espera = self.env.now
            yield req
            tiempos_espera_cola.append(self.env.now - tiempo_inicio_espera)
            yield self.env.process(servidor.procesar(solicitud))

    def seleccionar_servidor(self):
        servidor = self.servidores[self.ultimo_servidor]
        self.ultimo_servidor = (self.ultimo_servidor + 1) % len(self.servidores)
        return servidor


In [14]:
class BalanceadorDifuso:
    def __init__(self, env, servidores, grafo, umbral):
        self.env = env
        self.servidores = servidores
        self.grafo = grafo
        self.umbral = umbral

    def balancear(self, solicitud):
        tiempo_inicio_balanceo = self.env.now
        yield self.env.timeout(TIEMPO_BALANCEADOR)
        tiempos_espera_balanceador.append(self.env.now - tiempo_inicio_balanceo)

        servidor = self.seleccionar_servidor()
        with servidor.procesador.request() as req:
            tiempo_inicio_espera = self.env.now
            yield req
            tiempos_espera_cola.append(self.env.now - tiempo_inicio_espera)
            yield self.env.process(servidor.procesar(solicitud))

    def seleccionar_servidor(self):
        # selecciona un servidor con menor carga siguiendo el grafo y el umbral de carga
        servidor_inicial = random.choice(self.servidores) # aca por ahi podriamos hacer un round robin como dice en el paper
        return self.propagacion_difusa(servidor_inicial)

    def propagacion_difusa(self, servidor_actual):
        # busca el servidor menos cargado entre los vecinos si la diferencia de carga excede el umbral
        visitados = set()  # para evitar ciclos en el grafo
        cola = [servidor_actual]

        while cola:
            servidor = cola.pop(0)
            visitados.add(servidor.id_servidor)

            vecinos = [self.servidores[vid] for vid in self.grafo[servidor.id_servidor] if vid not in visitados]
            if not vecinos:
                break  # no hay vecinos a los que propagar

            servidor_menos_cargado = min(vecinos, key=lambda s: s.carga)
            diferencial_carga = servidor.carga - servidor_menos_cargado.carga

            if diferencial_carga > self.umbral:
                cola.append(servidor_menos_cargado)  # propaga al vecino menos cargado
            else:
                return servidor  # retorna el servidor actual si la carga es aceptable

        return servidor_actual  # devuelve el servidor inicial si no se encontró uno mejor

In [15]:
def generar_solicitudes(env, balanceador):
    solicitud_id = 0
    while True:
        yield env.timeout(random.expovariate(TASA_SOLICITUDES))
        env.process(balanceador.balancear(solicitud_id))
        solicitud_id += 1

In [16]:
def simular(tecnica, num_servidores, tiempo_simulacion):
    env = simpy.Environment()
    servidores = [Servidor(env, i) for i in range(num_servidores)]
       
    if tecnica == "Difuso":
        balanceador = BalanceadorDifuso(env, servidores, grafo, UMBRAL_PROPAGACION)
    elif tecnica == "Round Robin":
        balanceador = BalanceadorRoundRobin(env, servidores)
    else:
        raise ValueError("Técnica no soportada en esta simulación")

    env.process(generar_solicitudes(env, balanceador))
    env.run(until=tiempo_simulacion)

In [17]:
def mostrar_resultados():
    print("Tiempo promedio de espera en cola:", np.mean(tiempos_espera_cola))
    print("Tiempo promedio de espera en balanceador:", np.mean(tiempos_espera_balanceador))
    print("Tiempo promedio de atención en servidores:", np.mean(tiempos_atencion_servidores))
    print("Desviación estándar de espera en cola:", np.std(tiempos_espera_cola))
    print("Desviación estándar de atención en servidores:", np.std(tiempos_atencion_servidores))

In [18]:
print("Simulación Round Robin")
simular("Round Robin", NUM_SERVIDORES, TIEMPO_SIMULACION)
mostrar_resultados()

print("\nSimulación Difuso")
simular("Difuso", NUM_SERVIDORES, TIEMPO_SIMULACION)
mostrar_resultados()

Simulación Round Robin
Tiempo promedio de espera en cola: 0.00046404189185893165
Tiempo promedio de espera en balanceador: 0.010000000000001313
Tiempo promedio de atención en servidores: 0.10173624508149241
Desviación estándar de espera en cola: 0.007913338578366206
Desviación estándar de atención en servidores: 0.10294939399209758

Simulación Difuso
Tiempo promedio de espera en cola: 0.003677942292877666
Tiempo promedio de espera en balanceador: 0.01000000000000139
Tiempo promedio de atención en servidores: 0.09763979330849255
Desviación estándar de espera en cola: 0.02325543040304267
Desviación estándar de atención en servidores: 0.10096819883119065
